In [1]:
import os
import mne
import numpy as np
import joblib
from scipy import stats
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

In [2]:
from scipy.signal import find_peaks, welch
from scipy.stats import entropy, kurtosis, skew, iqr

def extract_features_multi(data, sfreq):
    feature_list = []
    count = 0
    
    # Zaman serisinde ozellik cikarimi yapiyorum. Buradaki ozniteliklerin cogunu TSFEL: Time Series Feature Extraction Library makalesinden aldim.

    for epoch in data:
        feats = []
        for ch_signal in epoch:
            t = np.arange(len(ch_signal))
            abs_energy = np.sum(ch_signal**2)                                                           # Sinyalin toplam enerjisi, guc olcusu
            area_curve = np.trapz(ch_signal)                                                            # Egri alti alan, sinyalin integraline benzer toplam degeri
            
            autocorr = np.corrcoef(ch_signal[:-1], ch_signal[1:])[0, 1] if np.std(ch_signal) > 0 else 0 # Sinyalin kendisiyle bir gecikmeyle olan korelasyonu
            
            avg_power = np.mean(ch_signal**2)                                                           # Ortalama guc
            centroid = np.sum(t * np.abs(ch_signal)) / (np.sum(np.abs(ch_signal)) + 1e-12)              # Sinyal agirlik merkezi
            
            hist, _ = np.histogram(ch_signal, bins=10, density=True)
            hist_entropy = entropy(hist + 1e-12)                                                        # Sinyal genlik dagiliminin duzensizligi
            
            freqs, psd = welch(ch_signal, sfreq, nperseg=min(256, len(ch_signal)))
            psd /= np.sum(psd) + 1e-12                                                                  # Frekanslara gore enerji dagilimi
            
            fund_freq = freqs[np.argmax(psd)] if len(psd) > 0 else 0                                    # En yuksek guce sahip frekans bileşeni
            
            hist_vals, hist_bins = np.histogram(ch_signal, bins=10)
            hist_mode = hist_bins[np.argmax(hist_vals)]                                                 # En cok gorulen genlik degeri
            
            human_energy = np.sum(psd[(freqs >= 0.5) & (freqs <= 40)])                                  # Insan EEG bandindaki enerji
            iqr_val = iqr(ch_signal)                                                                    # Sinyalin orta %50'sinin yayilimi
            kurt_val = kurtosis(ch_signal)                                                              # Carpiklik, uc deger yogunlugu
            
            max_val = np.max(ch_signal)                                                                 # Maksimum deger
            max_psd = np.max(psd)                                                                       # Maksimum guc
            max_freq = freqs[-1]                                                                         # En yuksek olculebilir frekans
            
            mean_val = np.mean(ch_signal)                                                               # Ortalama deger
            mad = np.mean(np.abs(ch_signal - mean_val))                                                 # Ortalama mutlak sapma
            mean_abs_diff = np.mean(np.abs(np.diff(ch_signal)))                                         # Komşu ornekler arasindaki ortalama mutlak fark
            mean_diff = np.mean(np.diff(ch_signal))                                                     # Komşu ornekler arasindaki ortalama fark
            
            med_val = np.median(ch_signal)                                                              # Medyan deger
            med_abs_dev = np.median(np.abs(ch_signal - med_val))                                        # Medyan mutlak sapma
            med_abs_diff = np.median(np.abs(np.diff(ch_signal)))                                        # Medyan komşu mutlak farki
            med_diff = np.median(np.diff(ch_signal))                                                    # Medyan komşu farki
            cumulative_power = np.cumsum(psd)
            med_freq = freqs[np.argmax(cumulative_power >= 0.5)] if len(psd) > 0 else 0                 # Medyan frekans
            
            min_val = np.min(ch_signal)                                                                 # Minimum deger
            
            diff_signal = np.diff(ch_signal)
            sign_changes = np.diff(np.sign(diff_signal))
            neg_turning = np.sum(sign_changes > 0)                                                      # Negatif egimden pozitif egime geciş sayisi
            pos_turning = np.sum(sign_changes < 0)                                                      # Pozitif egimden negatif egime geciş sayisi
            peaks, _ = find_peaks(ch_signal)
            n_peaks = len(peaks)                                                                        # Tepe sayisi
            peak_dist = np.mean(np.diff(peaks)) if n_peaks > 1 else 0                                   # Tepeler arasi ortalama mesafe
            
            if len(psd) > 0:
                peak_psd = np.max(psd)
                mask = psd >= (peak_psd / 2)
                try:
                    power_bandwidth = freqs[mask][-1] - freqs[mask][0]                                  # Gucun %50'sinden fazlasini iceren frekans araligi
                except:
                    power_bandwidth = 0
            else:
                power_bandwidth = 0
                
            rms = np.sqrt(avg_power)                                                                    
            signal_dist = np.sum(np.abs(ch_signal))                                                     # Mutlak genlik toplami
            skew_val = skew(ch_signal)                                                                  # Simetri olcusu
            slope = np.polyfit(t, ch_signal, 1)[0]                                                      # Lineer egim, trend
            
            spec_centroid = np.sum(freqs * psd) / (np.sum(psd) + 1e-12)                                 # Spektral agirlik merkezi
            spec_decrease = np.sum((psd[1:] - psd[0]) / np.arange(1, len(psd))) / (np.sum(psd[1:]) + 1e-12) if len(psd) > 1 else 0   # Yuksek frekanslardaki guc kaybi
            spec_ent = entropy(psd + 1e-12)                                                             # Spektral entropi
            spec_kurt = kurtosis(psd)                                                                   # Spektral carpiklik
            diff_psd = np.diff(psd)
            psd_sign_changes = np.diff(np.sign(diff_psd))
            spec_pos_turning = np.sum(psd_sign_changes < 0)                                             # Spektral donuş noktasi sayisi
            spec_roll_off = freqs[np.argmax(cumulative_power >= 0.85)] if len(psd) > 0 else 0           # Enerjinin %85'ine ulaşilan frekans
            spec_roll_on = freqs[np.argmax(cumulative_power >= 0.05)] if len(psd) > 0 else 0            # Enerjinin %5'ine ulaşilan frekans
            spec_skew = skew(psd)                                                                       # Spektral skewness
            spec_slope = np.polyfit(freqs, psd, 1)[0] if len(psd) > 1 else 0                            # Spektral egim
            spec_spread = np.sqrt(np.sum(psd * (freqs - spec_centroid)**2) / (np.sum(psd) + 1e-12))     # Spektral yayilma
            
            std = np.std(ch_signal)                                                                     # Standart sapma
            sum_abs_diff = np.sum(np.abs(np.diff(ch_signal)))                                           # Ornekler arasi toplam fark
            var = np.var(ch_signal)                                                                     # Varyans
            
            zero_crossings = np.sum(np.diff(np.sign(ch_signal)) != 0)
            zcr = zero_crossings / len(ch_signal)                                                       # Sinyalin kac kez sifiri gectigi
            
            features = [
                abs_energy, area_curve, autocorr, avg_power, centroid, hist_entropy,
                fund_freq, hist_mode, human_energy, iqr_val, kurt_val, max_val,
                max_psd, max_freq, mean_val, mad, mean_abs_diff, mean_diff, med_val,
                med_abs_dev, med_abs_diff, med_diff, med_freq, min_val, neg_turning,
                n_peaks, peak_dist, pos_turning, power_bandwidth, rms, signal_dist,
                skew_val, slope, spec_centroid, spec_decrease, spec_ent,
                spec_kurt, spec_pos_turning, spec_roll_off, spec_roll_on, spec_skew,
                spec_slope, spec_spread, std, sum_abs_diff, var, zcr
            ]
            
            feats.extend(features)
            
        feature_list.append(feats)
        count += 1
        
    print(f"ozellik cikarimi icin toplam {count} epoch işlendi.")
    return np.array(feature_list)

In [3]:
def preprocess_data(psg_file, hyp_file, epoch_duration=30.0):
    raw = mne.io.read_raw_edf(psg_file, preload=True, verbose=False)
    
    annotations = mne.read_annotations(hyp_file)                                                        # Hypnogram dosyasindan etiketleri cikartiyoruz.
    raw.set_annotations(annotations)
    
    stage_mapping = {                                                                                   # Olaylari mapliyoruz.
        'Sleep stage W': 0,
        'Sleep stage 1': 1,
        'Sleep stage 2': 2,
        'Sleep stage 3': 3,
        'Sleep stage 4': 3,
        'Sleep stage R': 4,
        'Sleep stage ?': -1,
        'Movement time': -1
    }
    
    events, event_dict = mne.events_from_annotations(                                                   # Epochlari ve etiketleri olusturuyoruz.
        raw, 
        event_id=stage_mapping,
        chunk_duration=epoch_duration
    )
    
    valid_events = [e for e in events if 0 <= e[2] <= 4]                                                # Gecerli uyku evrelerini filtreliyorum
    
    epochs = mne.Epochs(                                                                                # Epochlari olusturuyorum ve ozelliklerini cikartiyorum.
        raw, 
        valid_events, 
        tmin=0.0, 
        tmax=epoch_duration - 1/raw.info['sfreq'],
        baseline=None,
        preload=True
    )
    
    data = epochs.get_data()
    features = extract_features_multi(data, raw.info['sfreq'])
    labels = [e[2] for e in valid_events]
    
    return features, labels

In [4]:
def my_standard_scaler(X):              # Verilerimi 0-1 arasina standartize ediyorum.
    mean = np.mean(X, axis=0)           # Her sutunun ortalamasi
    std = np.std(X, axis=0)             # Her sutunun standart sapmasi
    X_scaled = (X - mean) / std         # Standardizasyon formulu
    return X_scaled

In [5]:
def predict_sleep_stages(psg_file, hyp_file):
    import joblib

    imputer = joblib.load('imputer.joblib')                                                 # Modelleri yukluyorum
    selector = joblib.load('selector.joblib')
    rf_model = joblib.load('random_forest_model.joblib')
    xgb_model = joblib.load('xgboost_model.joblib')
    mlp_model = load_model('mlp_model.h5')

    features, y_true = preprocess_data(psg_file, hyp_file)                                  # Ozellik cikarimi ve scaling yapiyorum.
    features_scaled = my_standard_scaler(features)
    features_imputed = imputer.transform(features_scaled)

    
    features_selected = selector.transform(features_imputed)                                # SelectKBest ile en iyi 150 feature'ı aliyorum.

    rf_pred = rf_model.predict(features_selected)                                           # Modellerle tahmin yapiyorum.
    xgb_pred = xgb_model.predict(features_selected)
    mlp_pred = np.argmax(mlp_model.predict(features_selected), axis=1)

    stage_map = {0: 'Wake', 1: 'N1', 2: 'N2', 3: 'N3', 4: 'REM'}                            # Performans ciktisi olusturuyorum.
    
    results = {
        'true': [stage_map[l] for l in y_true],
        'rf': [stage_map[p] for p in rf_pred],
        'xgb': [stage_map[p] for p in xgb_pred],
        'mlp': [stage_map[p] for p in mlp_pred]
    }
    
    return results

In [6]:
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score
def calculate_metrics(y_true, y_pred):                                                      # Performans verilerini elde ediyorum.
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    kappa = cohen_kappa_score(y_true, y_pred)
    return acc, f1, kappa

In [8]:
import glob
from sklearn.metrics import classification_report, confusion_matrix
if __name__ == "__main__":

    data_dir = "./SleepData/Sleep_EDF_Testing_Data"                                         # Test dosyamizi aliyorum.
    psg_files = glob.glob(os.path.join(data_dir, "*0-PSG.edf"))
    
    all_true = []                                                                           # Tum modeller icin toplu sonuclari tutuyorum.
    all_rf_pred = []
    all_xgb_pred = []
    all_mlp_pred = []
    
    tekVeriIcinCalistir=False                                                               # True oldugunda klasordeki sadece 1 veriyi okur ve test yapar. Exact bir veri icin asagidaki 0-PSG.edf'in ismini ozellestirebilirsiniz.
    for psg_file in psg_files:                                                              # Dosya ciftlerini seciyorum.
        hyp_file = psg_file.replace('0-PSG.edf', 'C-Hypnogram.edf')
        
        if not os.path.exists(hyp_file):
            print(f"Hipnogram dosyası bulunamadı: {hyp_file}")
            continue
        
        print(f"İşleniyor: {os.path.basename(psg_file)}")
        results = predict_sleep_stages(psg_file, hyp_file)
        
        all_true.extend(results['true'])
        all_rf_pred.extend(results['rf'])
        all_xgb_pred.extend(results['xgb'])
        all_mlp_pred.extend(results['mlp'])

        if (tekVeriIcinCalistir==True): break
                                                                                            # Performans ciktilarini  hesapliyorum.
    print("Random Forest Performansı:")
    print(classification_report(all_true, all_rf_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(all_true, all_rf_pred, labels=['Wake', 'N1', 'N2', 'N3', 'REM']))
    
    print("XGBoost Performansı:")
    print(classification_report(all_true, all_xgb_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(all_true, all_xgb_pred, labels=['Wake', 'N1', 'N2', 'N3', 'REM']))
    
    print("MLP Performansı:")
    print(classification_report(all_true, all_mlp_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(all_true, all_mlp_pred, labels=['Wake', 'N1', 'N2', 'N3', 'REM']))

    rf_metrics = calculate_metrics(all_true, all_rf_pred)                                   # Modellerin performansini karsilastiriyorum.
    xgb_metrics = calculate_metrics(all_true, all_xgb_pred)
    mlp_metrics = calculate_metrics(all_true, all_mlp_pred)

    print("MODEL PERFORMANS KARŞILAŞTIRMASI:")
    print(f"RF:    Accuracy={rf_metrics[0]:.4f}, F1={rf_metrics[1]:.4f}, Kappa={rf_metrics[2]:.4f}")
    print(f"XGB:   Accuracy={xgb_metrics[0]:.4f}, F1={xgb_metrics[1]:.4f}, Kappa={xgb_metrics[2]:.4f}")
    print(f"MLP:   Accuracy={mlp_metrics[0]:.4f}, F1={mlp_metrics[1]:.4f}, Kappa={mlp_metrics[2]:.4f}")

İşleniyor: SC4102E0-PSG.edf


C:\Users\EgeTERZI\AppData\Local\Temp\ipykernel_15204\3438200896.py:2: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_file, preload=True, verbose=False)
C:\Users\EgeTERZI\AppData\Local\Temp\ipykernel_15204\3438200896.py:2: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_file, preload=True, verbose=False)
C:\Users\EgeTERZI\AppData\Local\Temp\ipykernel_15204\3438200896.py:2: RuntimeWarning: Highpass cutoff frequency 16.0 is greater than lowpass cutoff frequency 0.7, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(psg_file, preload=True, verbose=False)
C:\Users\EgeTERZI\AppData\Local\Temp\ipykernel_15204\3438200896.py:5: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(annotations)


Used Annotations descriptions: [np.str_('Movement time'), np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage ?'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]
Not setting metadata
2857 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 2857 events and 3000 original time points ...
0 bad epochs dropped


C:\Users\EgeTERZI\AppData\Local\Temp\ipykernel_15204\2363082506.py:15: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area_curve = np.trapz(ch_signal)                                                            # Egri alti alan, sinyalin integraline benzer toplam degeri


ozellik cikarimi icin toplam 2857 epoch işlendi.


...

Used Annotations descriptions: [np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage 4'), np.str_('Sleep stage ?'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]
Not setting metadata
2814 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 2814 events and 3000 original time points ...
0 bad epochs dropped


C:\Users\EgeTERZI\AppData\Local\Temp\ipykernel_15204\2363082506.py:15: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area_curve = np.trapz(ch_signal)                                                            # Egri alti alan, sinyalin integraline benzer toplam degeri


ozellik cikarimi icin toplam 2814 epoch işlendi.
 1/88 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step

C:\Users\EgeTERZI\AppData\Local\Temp\ipykernel_15204\924106973.py:4: RuntimeWarning: invalid value encountered in divide
  X_scaled = (X - mean) / std         # Standardizasyon formulu
c:\Users\EgeTERZI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\impute\_base.py:637: UserWarning: Skipping features without any observed values: [ 13  60 107 147 154 163 178 194 201 216 225 226 241 248 257 263 272 281
 288 295 304 310 319]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Random Forest Performansı:
              precision    recall  f1-score   support

          N1       0.24      0.36      0.29       253
          N2       0.84      0.72      0.78      2465
          N3       0.74      0.64      0.69       519
         REM       0.79      0.60      0.68       938
        Wake       0.94      0.99      0.96      9602

    accuracy                           0.89     13777
   macro avg       0.71      0.66      0.68     13777
weighted avg       0.89      0.89      0.89     13777

Confusion Matrix:
[[9509   19   34    6   34]
 [  89   90   38    0   36]
 [ 379  126 1770  109   81]
 [  52    0  131  334    2]
 [ 122  135  122    0  559]]
XGBoost Performansı:
              precision    recall  f1-score   support

          N1       0.32      0.42      0.36       253
          N2       0.86      0.71      0.77      2465
          N3       0.74      0.66      0.70       519
         REM       0.82      0.62      0.70     